In [20]:
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

In [12]:
with rio.open("preprocessing_data/8bit/clipped.tif") as src:
    data = src.read()
    transform = src.transform

In [8]:
# This will get us the centroids of the clipped raster for every Pixel
centroids = []
x = []
y = []
for row in range(data.shape[1]):
    for col in range(data.shape[2]):
        centroids.append(Point(transform * (col+0.5, row+0.5)))
#         idx.append((col, row))
        x.append(col)
        y.append(row)
        

In [9]:
gdf_pt = gpd.GeoDataFrame(geometry = centroids)

In [10]:
gdf_pt['x'] = x
gdf_pt['y'] = y

In [83]:
gdf_pt

,geometry,x,y
0,POINT (-4.24337 5.42055),0,0
1,POINT (-4.24336 5.42055),1,0
2,POINT (-4.24336 5.42055),2,0
3,POINT (-4.24335 5.42055),3,0
4,POINT (-4.24335 5.42055),4,0
...,...,...,...
526819,POINT (-4.23909 5.41816),967,541
526820,POINT (-4.23909 5.41816),968,541
526821,POINT (-4.23908 5.41816),969,541
526822,POINT (-4.23908 5.41816),970,541


In [540]:
gdf_pt.drop(columns='idx', inplace = True)

In [542]:
# gdf_pt.to_file('data/preprocessing_data/centroid_raster.shp', driver="ESRI Shapefile", ignore_index = True)

# Dataset Creation

In [21]:
gdf = gpd.read_file("preprocessing_data/labels/OilPalm-tree-buffer.shp")

In [27]:
gdf['centroid'] = gdf['geometry'].centroid

/var/folders/83/ypn030093497vyfy0wz4lyqc0000gn/T/ipykernel_68722/1614308611.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


In [28]:
gdf

,id,geometry,centroid
0,1.0,"POLYGON ((-4.23940 5.42038, -4.23940 5.42037, ...",POINT (-4.23944 5.42038)
1,2.0,"POLYGON ((-4.23941 5.42023, -4.23941 5.42021, ...",POINT (-4.23945 5.42023)
2,3.0,"POLYGON ((-4.23933 5.42027, -4.23933 5.42026, ...",POINT (-4.23937 5.42027)
3,1.0,"POLYGON ((-4.23934 5.42042, -4.23935 5.42041, ...",POINT (-4.23938 5.42042)
4,1.0,"POLYGON ((-4.23927 5.42039, -4.23927 5.42038, ...",POINT (-4.23931 5.42039)
...,...,...,...
1277,NaN,"POLYGON ((-4.24093 5.41857, -4.24093 5.41855, ...",POINT (-4.24097 5.41857)
1278,NaN,"POLYGON ((-4.24087 5.41861, -4.24087 5.41860, ...",POINT (-4.24091 5.41861)
1279,NaN,"POLYGON ((-4.24093 5.41849, -4.24093 5.41848, ...",POINT (-4.24097 5.41849)
1280,NaN,"POLYGON ((-4.24094 5.41939, -4.24094 5.41938, ...",POINT (-4.24098 5.41939)


# Finding Closest Point to centroids

In [29]:
closest = []
for x, y in zip(gdf["centroid"].x, gdf["centroid"].y):
    ed = np.sqrt((gdf_pt.geometry.x - x)**2 + (gdf_pt.geometry.y - y)**2)
    idx = np.argmin(ed)
    closest.append(idx)

In [31]:
gdf_pt.iloc[closest, :].to_file("preprocessing_data/inter/closest_point.shp", driver="ESRI Shapefile", ignore_index= True)

In [32]:
# gdf_pt[gdf_pt.iloc[closest, :].geometry.y < 5.4183013  ]
closest_pt = gdf_pt.iloc[closest, :]

# Dividing the dataset into patches

In [39]:
###
###
### This is done through qgis
### the clipped image is divided into 256x256 patches

# Removing the edge labels

In [82]:
import glob
import os
for i in glob.glob("preprocessing_data/8bit/patched/*.tif"):
    print((os.path.split(i)))

    with rio.open(i) as src:
        data_patch = src.read()
        data_transform = src.profile
        
    file_name = os.path.splitext(os.path.split(i)[1])[0]
    
    lower_ext = data_transform['transform'] * (9,9)
    higher_ext = data_transform['transform'] * (data_patch.shape[2]-9,data_patch.shape[1]-9)
    closest_pt[(closest_pt.geometry.x > lower_ext[0]) & (closest_pt.geometry.x <= higher_ext[0]) & (closest_pt.geometry.y <= lower_ext[1]) & (closest_pt.geometry.y >= higher_ext[1])].to_file(f"preprocessing_data/inter/patch_shp/{file_name}.shp", driver="ESRI Shapefile")    

('preprocessing_data/8bit/patched', '8bit.6.tif')
('preprocessing_data/8bit/patched', '8bit.7.tif')
('preprocessing_data/8bit/patched', '8bit.5.tif')
('preprocessing_data/8bit/patched', '8bit.4.tif')
('preprocessing_data/8bit/patched', '8bit.0.tif')
('preprocessing_data/8bit/patched', '8bit.1.tif')
('preprocessing_data/8bit/patched', '8bit.3.tif')
('preprocessing_data/8bit/patched', '8bit.2.tif')
